In [1]:
import pandas as pd
data = pd.read_csv("clase.csv").sample(1000)
print(f"Tenemos {data.shape[0]:,d}documentos.")
data.head()

Tenemos 1,000documentos.


,Article
486,The FCC has unveiled a proposal that would res...
109,"This time last year, Stephanie Johnson was mis..."
961,"This story is part of NPR’s series, Break It..."
697,This is the second story in a series. Read P...
658,"A few ”short” years ago, during my sophomore y..."


Se recomienda buscar el origen de las palabras 

In [2]:
import re
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [3]:
def pre_procesado(texto):
     texto = texto.lower()
     texto = re.sub(r"[\W\d_]+", " ", texto)
     texto = [palabra for palabra in texto.split() if palabra not in stopwords]
     return texto
 
data['Pre-Processed'] = data['Article'].apply(lambda texto: pre_procesado(texto))
data.head()

,Article,Pre-Processed
486,The FCC has unveiled a proposal that would res...,"[fcc, unveiled, proposal, would, restrict, int..."
109,"This time last year, Stephanie Johnson was mis...","[time, last, year, stephanie, johnson, miserab..."
961,"This story is part of NPR’s series, Break It...","[story, part, npr, series, break, bernie, sand..."
697,This is the second story in a series. Read P...,"[second, story, series, read, part, annual, ki..."
658,"A few ”short” years ago, during my sophomore y...","[short, years, ago, sophomore, year, city, col..."


In [4]:
import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from pprint import pprint

C:\ProgramData\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [5]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(data['Pre-Processed'].values)
dictionary

In [15]:
# Filtrar palabras muy frecuentes y poco frecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]
# Train the topic model 
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=7, passes=100)

In [17]:
model.print_topics(7)

[(0,
  '0.022*"trump" + 0.009*"clinton" + 0.007*"campaign" + 0.005*"think" + 0.005*"sanders" + 0.004*"even" + 0.004*"voters" + 0.004*"donald" + 0.004*"students" + 0.004*"vote"'),
 (1,
  '0.009*"think" + 0.008*"company" + 0.006*"companies" + 0.005*"make" + 0.005*"going" + 0.005*"well" + 0.004*"media" + 0.004*"information" + 0.004*"want" + 0.004*"cia"'),
 (2,
  '0.012*"u" + 0.005*"world" + 0.005*"percent" + 0.005*"trade" + 0.004*"country" + 0.004*"year" + 0.004*"government" + 0.004*"president" + 0.004*"could" + 0.004*"nuclear"'),
 (3,
  '0.007*"food" + 0.006*"u" + 0.005*"water" + 0.005*"city" + 0.004*"isis" + 0.004*"war" + 0.004*"north" + 0.003*"two" + 0.003*"country" + 0.003*"back"'),
 (4,
  '0.004*"life" + 0.004*"know" + 0.004*"get" + 0.004*"way" + 0.004*"world" + 0.004*"really" + 0.003*"story" + 0.003*"think" + 0.003*"even" + 0.003*"black"'),
 (5,
  '0.013*"health" + 0.007*"school" + 0.006*"care" + 0.005*"get" + 0.005*"percent" + 0.005*"children" + 0.004*"students" + 0.004*"women" + 0

In [18]:
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

In [19]:
def get_doc_top_n(text_processed, n):

     d = dictionary.doc2bow(text_processed)
     topics = dict(model.get_document_topics(d))
     try:
         return topic[n]
     except:
         return None

In [20]:
d = dictionary.doc2bow(["trump", "clinton", "white", "house"])
topics = model.get_document_topics(d)
topics

[(0, 0.8282746),
 (1, 0.028573211),
 (2, 0.028592804),
 (3, 0.02861301),
 (4, 0.028594792),
 (5, 0.028573364),
 (6, 0.028778195)]

In [21]:
def get_doc_top_n(text_processed, n):
     d = dictionary.doc2bow(text_processed)
     topics = dict(model.get_document_topics(d))
     try:
         return topics[n]
     except:
         return None

In [23]:
for t in range(0,7):
     top_name = f"topic_{t}"
     data[top_name] = data['Pre-Processed'].apply(lambda doc: get_doc_top_n(doc,t))
data

,Article,Pre-Processed,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
644,The first presidential debate between Donald T...,"[first, presidential, debate, donald, trump, h...",0.581397,NaN,0.154004,NaN,NaN,NaN,0.253100
652,Updated on Jan. 27 to add video of speech: Idr...,"[updated, jan, add, video, speech, idris, elba...",0.386808,NaN,0.091631,0.018061,0.501396,NaN,NaN
437,After hearing oral arguments on what could be ...,"[hearing, oral, arguments, could, one, importa...",NaN,NaN,NaN,NaN,0.047556,0.133624,0.816047
247,Nearly 1 in 5 children each year suffers a psy...,"[nearly, children, year, suffers, psychiatric,...",NaN,NaN,NaN,NaN,NaN,0.998032,NaN
126,"Surrounded by shouting, he’s completely silent...","[surrounded, shouting, completely, silent, chi...",NaN,NaN,NaN,0.392010,0.352859,0.253109,NaN
...,...,...,...,...,...,...,...,...,...
688,We make dozens of decisions on a daily basis: ...,"[make, dozens, decisions, daily, basis, breakf...",NaN,0.764825,0.048495,NaN,NaN,0.185032,NaN
781,"For months, FBI Director James Comey has been ...","[months, fbi, director, james, comey, warning,...",NaN,NaN,NaN,NaN,NaN,0.172840,0.819911
528,When Jewish couple Mikey Franklin and Sonya Sh...,"[jewish, couple, mikey, franklin, sonya, shpil...",0.045340,NaN,0.063150,NaN,0.576320,0.071065,0.243714
248,Hillary Clinton and Donald Trump this week vis...,"[hillary, clinton, donald, trump, week, visite...",0.703194,NaN,NaN,NaN,0.293190,NaN,NaN


In [24]:
for t in range(0,7):
     print(f"****************************** TOPIC {t}******************************")
     for i,row in enumerate(data.sort_values(f"topic_{t}", ascending=False)['Article'].head()):
         print(f"Article #{i+1}")
         print(row[:500])
         print()
     print()

****************************** TOPIC 0******************************
Article #1
With every state that voted in February, the contours of the 2016 presidential election changed. Iowa, New Hampshire and Nevada all transformed the landscape in both parties. On Saturday night, in South Carolina, the Earth moved once again. Hillary Clinton won, as expected, but the breadth and depth of her victory were breathtaking. She prevailed by more than 47 percentage points in the most populous state to vote thus far, winning by more than twice the margin of her loss to Bernie Sanders in N

Article #2
To understand how Bernie Sanders’ presidential campaign is reacting to the candidate’s gigantic loss in South Carolina’s Democratic primary, it’s important to understand how he reacted to another loss just a few weeks ago, in Iowa. The two results couldn’t be more different. Sanders’ performance in Iowa could be considered a virtual tie with Hillary Clinton, and he called it just that. Clinton won the De